In [ ]:
# !pip install llm=="0.12" chromadb=="0.4.17" watchdog=="3.0.0" supabase pinecone-client
# !llm install llm-clip

In [3]:
import llm
import glob
import base64
import hashlib
import chromadb

In [4]:
from supabase import create_client
from google.colab import userdata

def get_image_url(file_name):
  path = "city_furniture/" + file_name
  url = userdata.get("SUPABASE_URL")
  key = userdata.get("SUPABASE_KEY")
  BUCKET_NAME = "image-bin"

  # Client
  supabase_client = create_client(url, key)
  url = supabase_client.storage.from_(BUCKET_NAME).get_public_url(path)
  return url

In [ ]:
# get_image_url("Alden Black Nightstand.jpg")

'https://kxzrpvbbccaxzwlcalcy.supabase.co/storage/v1/object/public/image-bin/city_furniture/Alden Black Nightstand.jpg?'

In [7]:
## Function to use Fetch request
import requests
def fetch(task_id):

  endpoint = "https://api.midjourneyapi.xyz/mj/v2/fetch"

  data = {
      "task_id": task_id
  }

  response = requests.post(endpoint, json=data)

  return response.json()

In [16]:
## Generate Embeddings

images = glob.glob("/content/drive/MyDrive/City_Furniture/*.jpg")

model = llm.get_embedding_model("clip")

embeddings = []
for image in images:
  with open(image, "rb") as image_file:
    embedding = model.embed(image_file.read())
    file_name = image.split("/")[-1]
    image_url = get_image_url(file_name)
    image_url = image_url.replace(" ","%20")
    embeddings.append({
      "embedding": embedding,
      "filePath": image,
      "image_url": image_url,
      "id": base64.b64encode(hashlib.sha256(image.encode()).digest()).decode()
    })

In [ ]:
embeddings[0]

In [22]:
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata

# Initialize Pinecone client
pinecone_api_key = userdata.get("PINECONE_API_KEY")
index_name = "test2"

pc = Pinecone(api_key=pinecone_api_key)

# Now do stuff
if index_name not in pc.list_indexes().names():
  pc.create_index(
            name=index_name,
            dimension=512,
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )
index = pc.Index(index_name)
# Add embeddings and metadata to the index
for e in embeddings:
    index.upsert(vectors=[
        {
            "id":e["id"],
            "values": e["embedding"],
            "metadata": {
            "filePath": e["filePath"],
            "image_url": e["image_url"]
                      }
        },
    ])

In [23]:
## Query the embedding

query_embedding = model.embed("A large maplewood bed")
index_response = index.query(
    vector=query_embedding,
    top_k=1,
    includeMetadata= True
)


In [24]:
index_response

{'matches': [{'id': 'dhJEOD7pmPfwnHQSSFfJKAUjFlFluVFglh9gZ8rX7YA=',
              'metadata': {'filePath': '/content/drive/MyDrive/City_Furniture/Cortina '
                                       'White Uph Platform Bed.jpg',
                           'image_url': 'https://kxzrpvbbccaxzwlcalcy.supabase.co/storage/v1/object/public/image-bin/city_furniture/Cortina%20White%20Uph%20Platform%20Bed.jpg?'},
              'score': 141.443253,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [25]:
index.describe_index_stats()

{'dimension': 512,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 20}},
 'total_vector_count': 20}

In [27]:
import requests
from google.colab import userdata
import time

# Function to generate Image based on the user query

def image_generation(image_url, user_query):
  prompt = f"{image_url} {user_query}"

  X_API_KEY = userdata.get("X_API_KEY")

  endpoint = "https://api.midjourneyapi.xyz/mj/v2/imagine"

  headers = {
      "X-API-KEY": X_API_KEY
  }

  data = {
      "prompt": prompt,
      "process_mode": "fast",
  }

  response = requests.post(endpoint, headers=headers, json=data)
  image_gen_req_output = response.json()
  task_id = image_gen_req_output["task_id"]
  time.sleep(60)

  ##### FETCH for Image ######
  fetch_response = fetch(task_id)
  while fetch_response["status"] == "processing":
    time.sleep(20)
    fetch_response = fetch(task_id)
  image = fetch_response["task_result"]["image_url"]
  return image

In [28]:
def main():
  query = input("Enter your query: ")
  query_embedding = model.embed(query)
  index_response = index.query(
      vector=query_embedding,
      top_k=1,
      includeMetadata= True
  )

  data = index_response["matches"][0]["metadata"]

  image_url = data["image_url"].replace(" ","%20")

  url = image_generation(image_url, query)

  return url

In [31]:
main()

Enter your query: A large bench


'https://img.midjourneyapi.xyz/mj/cfcb5620-be84-4253-9130-767b6e8c63fc.png'

## Test Results


<h4>Query: A black nightstand</h4>
<img src="https://img.midjourneyapi.xyz/mj/c5e34671-98af-46a7-8d2b-a468d486a832.png">


<h4>Query: A large maplewood bed </h4>
<img src="https://img.midjourneyapi.xyz/mj/0b7d043d-011d-4a96-9115-f7765cf24a84.png">

<h4>Query: A large bench </h4>
<img src="https://img.midjourneyapi.xyz/mj/cfcb5620-be84-4253-9130-767b6e8c63fc.png">